## ПЯТОЕ ДОМАШНЕЕ ЗАДАНИЕ. 
## *LU и LUP разложения. Решение СЛАУ*

### Выполнила: Леонтьева Ксения Андреевна
### Группа: НПМбд-01-19

In [246]:
import numpy as np
import scipy.linalg

## LU - РАЗЛОЖЕНИЕ

**LU - разложение матрицы А** - это представление этой матрицы в виде произведения A = LU, где L - нижняя треугольная матрица, U - верхняя треугольная ступенчатая матрица.

Для реализации данного разложения необходима единичная матрица, которую мы мысленно приписываем к нашей исходной матрице с правой стороны и далее применяем метод Гаусса для этих матриц (параллельно). В данном случае исходная матрица обозначается U, а единичная - L.

Дополнительно введем еще одну единичную матрицу M, которая будет хранить коэффициенты от деления первого элемента под главной диагональю на элемент главной диагонали, взятые в противоположным знаком. Чтобы не вводить два цикла будем брать деление не поэлементно, а делить ту часть столбца (вектор), который находится под элементом главной диагонали на этот элемент (см. #1).

Затем нужно изменять наши исходную и единичную матрицы соответственно. Для этого необходимо прибавить сразу ко всем строчкам, которые находятся под строчкой (U[i]), содержащей текущий диагональный элемент, произведение этой строчки (U[i]) на транспонированную матрицу коэффициентов (часть матрицы M) для данного столбца (т.о., это произведение (поэлементное!) будет представлять собой матрицу той же размерности, что и матрица, содержащая ранее указанные "все строчки, находящиеся под строчкой с текущим диагональным элементом") (см. #2). Сложение и вычитание матриц также поэлементные. Единичная матрица преобразовывается аналогичным образом.

В результате чего, мы получим следующее соотношение между матрицами: LA = U. Чтобы прийти в соотношению A = LU, нужно получить из текущей матрицы L обратную (т.к. текущая квадратная матрица L невырожденная, то она имеет квадратную матрицу) (см. #3).

In [247]:
def LU_decomposition(A):
    U = np.copy(A)
    n = len(U)
    M = np.eye(n)
    L = np.eye(n)
    for i in range (len(U)-1): 
        M[i+1:,i] = - U[i+1:,i] / U[i][i]                                 #1
        U[i+1:,:] = U[i] * np.transpose([M[i+1:,i],]*1) + U[i+1:,:]       #2
        L[i+1:,:] = L[i] * np.transpose([M[i+1:,i],]*1) + L[i+1:,:]
    L = np.linalg.inv(L)                                                  #3
    return L, U

In [248]:
TestMatrix1 = np.array([[1.,2,3],[4,5,6],[7,8,10]])
L1, U1 = LU_decomposition(TestMatrix1)
print('Матрица L')
print(L1)
print('Матрица U')
print(U1)

Матрица L
[[ 1. -0. -0.]
 [ 4.  1. -0.]
 [ 7.  2.  1.]]
Матрица U
[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  1.]]


## Решаем уравнение L  y = b

Так как матрица L ступенчатая с единицамим на главной диагонали, то для начала мы можем сразу найти нулевой элемент вектора y (в программе имена для y и b совпадают), затем каждый раз из правой части уравнения вычитать произведение соответствующей строки из матрицы L на вектор y (обозначенный как b, b[:i] будет давать для 1 строки элемент y0 (b0), а для 2 строки - y0 (b0) и y1 (b1)).

In [249]:
def Solve_Ly_equals_b(L, b):
    b[0] = b[0] / L[0][0]
    for i in range(1,len(b)):
        b[i] = b[i] - np.dot(L[i][:i], b[:i])
    return(b)

In [250]:
TestVectorb1 = [5,2,3]
y1 = Solve_Ly_equals_b(L1, TestVectorb1)
print(y1)

[5.0, -18.0, 4.0]


## Решаем уравнение U  x = y

Решение данного уравнения аналогично решению предыдущего уравнения, только в данном случае, т.к. матрица U - верхняя треугольная, мы находим решение, идя снизу вверх, от строки с наибольшим индексом к строке с наименьшим индексом. При этом разность делим на соответствующий диагональный элемент, потому что в матрице U на диагонали не единицы, а различные числа.

In [251]:
def Solve_Ux_equals_y_for_LU(U, y):
    y[-1] = y[-1] / U[-1][-1]
    for i in range(len(y)-2, -1, -1):
        y[i] = (y[i] - np.dot(U[i][i+1:], y[i+1:])) / U[i][i]
    return y

In [252]:
TestAnswer1 = Solve_Ux_equals_y_for_LU(U1,y1)
print('Ответ:')
print(TestAnswer1)

Ответ:
[-3.0, -2.0, 4.0]


## Итоговая функция (ответ для решения СЛАУ методом LU-разложения)

In [253]:
def SOLVE_LU(U, b):
    L, U = LU_decomposition(U)
    y = Solve_Ly_equals_b(L,b)
    x = Solve_Ux_equals_y_for_LU(U,y)
    return x

In [254]:
Matrix_for_LU = np.array([[1.,2,3],[4,5,6],[7,8,10]])
b1 = [5,2,3]

Answer_LU = SOLVE_LU(Matrix_for_LU,b1)
print('Ответ:')
print(Answer_LU)

Ответ:
[-3.0, -2.0, 4.0]


## LUP - РАЗЛОЖЕНИЕ

**LUP - разложение матрицы А** - это представление этой матрицы в виде PA = LU, где L - нижняя треугольная матрица, U - верхняя треугольная ступенчатая матрица, P - матрица перестановок.

Нахождение данного разложения осуществляется по аналогичному алгоритму, что и для LU-разложения, но при этом нам нужно менять строчки местами (выбираем наибольший по модулю ведущий элемент) и также ввести матрицу перестановок P, в которой строчки будут меняться параллельно с изменением матрицы U.

Для нахождения максимального по модулю ведущего элемента мы используем дополнительный массив С, который хранит индексы максимальных по модулю ведущих элементов в столбцах для конкретных матриц U[i:,i:], а чтобы выбрать из этого массива максимальный ведущий элемент по модулю для всей матрицы U, мы должны обращаться к нулевому элементу этого массива "плюс" текущее значение i. Если эта сумма больше текущего значения i, то меняем местами строки.

#### Как работает массив С:

In [255]:
Z = np.array([[1.,2,3],[4,-5,6],[7,8,10]])
q = len(Z)
N = np.eye(q)
for i in range(q-1):
    V = abs(Z[i:,i:])
    print('Исходная матрица')
    print(Z)
    print('')
    print('Модуль среза исходной матрицы')
    print(V)
    print('')
    С = np.argmax(V, axis = 0)
    print('Массив С (выводит индексы максимальных по модулю элементов в каждом столбце среза)')
    print(С)
    print('')
    print('Индекс максимального по модулю элемента в исходной матрице')
    print(С[0] + i)
    print('')
    if С[0] + i > i:
        
        d = np.copy(Z[i])
        Z[i] = Z[С[0]+i]
        Z[С[0]+i] = d

    N[i+1:,i] = - Z[i+1:,i] / Z[i][i]
    Z[i+1:,:] = Z[i] * np.transpose([N[i+1:,i],]*1) + Z[i+1:,:]


Исходная матрица
[[ 1.  2.  3.]
 [ 4. -5.  6.]
 [ 7.  8. 10.]]

Модуль среза исходной матрицы
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8. 10.]]

Массив С (выводит индексы максимальных по модулю элементов в каждом столбце среза)
[2 2 2]

Индекс максимального по модулю элемента в исходной матрице
2

Исходная матрица
[[ 7.          8.         10.        ]
 [ 0.         -9.57142857  0.28571429]
 [ 0.          0.85714286  1.57142857]]

Модуль среза исходной матрицы
[[9.57142857 0.28571429]
 [0.85714286 1.57142857]]

Массив С (выводит индексы максимальных по модулю элементов в каждом столбце среза)
[0 1]

Индекс максимального по модулю элемента в исходной матрице
1



## Итоговый код для LUP-разложения

In [256]:
def LUP_decomposition(A):
    U = np.copy(A)
    n = len(U)
    M = np.eye(n)
    L = np.eye(n)
    P = np.eye(n)
    for i in range(n-1):
        V = abs(U[i:,i:])
        С = np.argmax(V, axis = 0)

        if С[0] + i > i:

            d = np.copy(U[i])
            U[i] = U[С[0]+i]
            U[С[0]+i] = d

            d = np.copy(L[i])
            L[i] = L[С[0]+i]
            L[С[0]+i] = d

            d = np.copy(P[i])
            P[i] = P[С[0]+i]
            P[С[0]+i] = d

        M[i+1:,i] = - U[i+1:,i] / U[i][i]
        U[i+1:,:] = np.round(U[i] * np.transpose([M[i+1:,i],]*1) + U[i+1:,:],8)
        L[i+1:,:] = L[i] * np.transpose([M[i+1:,i],]*1) + L[i+1:,:]
    L = np.linalg.inv(L)  
    L = np.dot(P,L)
    return L, U, P

In [257]:
TestMatrix2 = np.array([[1.,2,3],[4,5,6],[7,8,10]])
L2, U2, P2 = LUP_decomposition(TestMatrix2)
print('Матрица L')
print(L2)
print('')
print('Матрица U')
print(U2)
print('')
print('Матрица P')
print(P2)
print('')

Матрица L
[[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [0.57142857 0.5        1.        ]]

Матрица U
[[ 7.          8.         10.        ]
 [ 0.          0.85714286  1.57142857]
 [ 0.          0.         -0.5       ]]

Матрица P
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]



#### Проверим LUP-разложение с помощью библиотеки scipy:

In [258]:
A, B, C = scipy.linalg.lu(np.array([[1.,2,3],[4,5,6],[7,8,10]]))
print('Матрица L')
print(B)
print('')
print('Матрица U')
print(C)
print('')
print('Матрица P')
print(A)
print('')
print('Матрица P транспонированная')
print(np.transpose(A))
print('')

Матрица L
[[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [0.57142857 0.5        1.        ]]

Матрица U
[[ 7.          8.         10.        ]
 [ 0.          0.85714286  1.57142857]
 [ 0.          0.         -0.5       ]]

Матрица P
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

Матрица P транспонированная
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]



## Решаем уравнение L y = P b

Решение аналогично решению уравнения L * y = b, только предварительно нужно домножить правую часть еще на матрицу P. 

In [259]:
def Solve_Ly_equals_Pb(L, P, b):
    right = np.dot(P, b)
    right[0] = right[0] / L[0][0]
    for i in range(1,len(right)):
        right[i] = (right[i] - np.dot(L[i][:i], right[:i])) / L[i][i]
    return(right)

In [260]:
TestVectorb2 = [5,2,3]
y2 = Solve_Ly_equals_Pb(L2, P2, TestVectorb2)
print(y2)

[ 3.          4.57142857 -2.        ]


#### Проверим решение этого уравнения с помощью библиотеки scipy

In [261]:
Y = scipy.linalg.solve(L2, np.dot(P,b))
print(Y)

[ 3.          4.57142857 -2.        ]


## Решаем уравнение U x = y

Решение аналогично решению уравнения U * x = y при LU-разложении.

In [262]:
def USolve_Ux_equals_y_for_LUP(y, U):
    y[-1] = y[-1] / U[-1][-1]
    for i in range(len(y)-2, -1, -1):
        y[i] = (y[i] - np.dot(U[i][i+1:], y[i+1:])) / U[i][i]
    return y

In [263]:
TestAnswer1 = USolve_Ux_equals_y_for_LUP(y2, U2)
print('Ответ:')
print(np.round(TestAnswer1))

Ответ:
[-3. -2.  4.]


#### Проверим решение этого уравнения с помощью библиотеки scipy

In [264]:
W = scipy.linalg.solve(U2,Y)
print(np.round(W))

[-3. -2.  4.]


## Итоговая функция (ответ для решения СЛАУ методом LUP-разложения)

In [265]:
def SOLVE_LUP(U, b):
    L, U, P = LUP_decomposition(U)
    y = Solve_Ly_equals_Pb(L,P,b)
    x = USolve_Ux_equals_y_for_LUP(y,U)
    return x

In [266]:
Matrix_for_LUP = np.array([[1.,2,3],[4,5,6],[7,8,10]])
b2 = [5,2,3]

Answer_LUP = SOLVE_LUP(Matrix_for_LUP,b2)
print('Ответ:')
print(np.round(Answer_LUP))

Ответ:
[-3. -2.  4.]


## Вывод

LU-разложение для произвольной квадратной невырожденной матрицы возможно не всегда, необходимо, чтобы в процессе решения не было деления на ноль, в противном случае эффективнее использовать LUP-разложение, отличие которого в том, что перед каждым шагом нахождения коэффициентов, необходимых для последующего сложения строк, присутствует операция перестановки местами двух строк соответствующей матрицы (с выбором наибольшего по модулю ведущего элемента), обеспечивающая получение ненулевых ведущих элементов.

Например:

In [267]:
Matrix1 = np.array([[0,2,3],[4,5,6],[7,8,10]])

In [268]:
 L_test1, U_test1 = LU_decomposition(Matrix1)

<ipython-input-247-64894099cbad>:7: RuntimeWarning: divide by zero encountered in true_divide
  M[i+1:,i] = - U[i+1:,i] / U[i][i]                                 #1
<ipython-input-247-64894099cbad>:8: RuntimeWarning: invalid value encountered in multiply
  U[i+1:,:] = U[i] * np.transpose([M[i+1:,i],]*1) + U[i+1:,:]       #2
<ipython-input-247-64894099cbad>:9: RuntimeWarning: invalid value encountered in multiply
  L[i+1:,:] = L[i] * np.transpose([M[i+1:,i],]*1) + L[i+1:,:]


In [269]:
 L_test2, U_test2, P_test = LUP_decomposition(Matrix1)